In [1]:
import requests
import pandas as pd
# Library to extract the statistics data 
import cbsodata
import json
import os


from ipywidgets import VBox, HBox, IntSlider, Layout, jslink, Play
from pathlib import Path
from typing import Dict, List
from bqplot import Mercator, Map, topo_load, Figure, \
    Tooltip, ColorScale, ColorAxis, Lines, LinearScale, Axis


PROVINCE_ID = int
NORMALIZED_INCOME = float

In [2]:
# Input Netherlands map is from https://www.webuildinternet.com/articles/2015-07-19-geojson-data-of-the-netherlands/provinces.geojson

def add_feature_id(input_geojson: Path, output_geojson: Path) -> Dict[int, str]:
    """
    Add id for every feature of a geojson file and 
    Return a dict of id and its feature name
    """

    with open(input_geojson, "rt") as in_f:
        features = []
        id_to_name = {}
        input_geodict = json.load(in_f)
        for idx, feature in enumerate(input_geodict['features'], start=1):
            feature["id"] = idx
            id_to_name[idx] = feature['properties']['name']
            features.append(feature)
            
    with open(output_geojson, "wt") as out_f:
        json.dump({'features': features}, out_f)
        
    return id_to_name
                    
province_id_to_name = add_feature_id(input_geojson="data/provinces.geojson",
                                     output_geojson="data/provinces_with_id.geojson")
province_id_to_name
province_name_to_id =  {v: k for k, v in province_id_to_name.items()}

In [3]:
def get_nl_provinces(province_geojson: Path) -> List[str]:
    with open(province_geojson, "rt") as in_f:
        province_dict = json.load(in_f)
        
    provinces = [province_feature['properties']['name'] 
                 for province_feature in province_dict['features']]
    return provinces


PROVINCE_GEOJSON = Path.cwd() / "data/provinces_with_id.geojson"
NL_PROVINCES = get_nl_provinces(PROVINCE_GEOJSON)

In [4]:
def get_income_data(data_id:str, output_file: Path) -> pd.DataFrame:
    if Path(output_file).exists() and os.path.getsize(str(output_file)):
        return pd.read_csv(output_file)
    
    print(f"Downloading data {data_id}")
    inkomen_data = pd.DataFrame(cbsodata.get_data(data_id))
    province_data = inkomen_data[inkomen_data['Region'].isin(NL_PROVINCES)]
    assert len(province_data["Region"].unique()) == 12 # There are 12 provinces in NL
    households_total = province_data[province_data["Households"] == 'Households, total']

    income = households_total[["Region", "Period", "MixedIncomeNet_1"]]
    # Assign province_id
    income = income.assign(province_id=income["Region"].map(province_name_to_id))
    income["Period"] = income["Period"].astype(int)

    income = income.rename(columns={'Region': 'province_name', 
                                    'Period': 'period', 
                                    'MixedIncomeNet_1': 'income'})

    with open(output_file, "wt") as out_f:
        income.to_csv(out_f, index=False)
        
    return income
    

In [5]:
INCOME_DATA = get_income_data("71103ENG", "data/income.csv")

In [6]:
# INCOME_DATA.head()

In [7]:
YEARLY_INCOME = INCOME_DATA.pivot(index='province_id', columns='period')[['income']]
YEARLY_INCOME.columns = YEARLY_INCOME.columns.droplevel()

# YEARLY_INCOME.head()

In [8]:
def get_id_to_income(year: int) -> Dict[PROVINCE_ID, NORMALIZED_INCOME]:
    is_within_year = INCOME_DATA["period"] == year
    yearly_income = INCOME_DATA[is_within_year]
    normalized_income = yearly_income["income"] / sum(yearly_income["income"])
    id_to_income = dict(zip(yearly_income["province_id"], normalized_income))
    return id_to_income
    
    

In [9]:
most_recent_id_to_income = get_id_to_income(1995)
# most_recent_id_to_income

In [10]:
# Generate income distribution per province in NL

sc_geo = Mercator(scale_factor=7000, center=(5.5, 53.0))
tooltip = Tooltip(fields=['name'], labels=['province'])
col_scale = ColorScale()
ax_col = ColorAxis(scale=col_scale, label="ratio", visible=True)
nl_map = Map(map_data=topo_load(PROVINCE_GEOJSON),
             scales={'projection': sc_geo, 'color': col_scale},
             colors={'default_color': 'Grey'},
             color=most_recent_id_to_income,
             tooltip=tooltip,
             interactions={'click': 'select', 'hover': 'tooltip'},
             visible=True)
nl_fig = Figure(marks=[nl_map], axes=[ax_col], title='Wealth of The Netherlands',
               layout={
                   'min_width': '400px', 'width': 'auto',
                   'min_height': '400px', 'width': 'auto'
               })


def plot_yearly_income_per_province(*_):
    if nl_map.selected is not None and len(nl_map.selected) > 0:
        selected_provinces = nl_map.selected
        province_yearly_income_plot.y = YEARLY_INCOME.loc[selected_provinces]
        province_yearly_income_plot.labels = [province_id_to_name[province_id] 
                                      for province_id in selected_provinces]
    else:
        province_yearly_income_plot.labels = []
        province_yearly_income_plot.y = []
  
    
nl_map.observe(plot_yearly_income_per_province, 'selected')


sc_x = LinearScale()
sc_y = LinearScale()
ax_x = Axis(scale=sc_x, label='Index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='Thousands (Euro)')
province_yearly_income_plot = Lines(x=list(YEARLY_INCOME.columns), y=[],
                                    scales={'x': sc_x, 'y': sc_y},
                                 
                                    display_legend=True)
province_yearly_income_fig = Figure(marks=[province_yearly_income_plot], 
                                    axes=[ax_x, ax_y], title='Income per Year',
                                    layout={
                                        'min_width': '400px', 'width': 'auto',
                                        'min_height': '400px', 'height': 'auto'
                                    })


year_slider = IntSlider(description='year', min=1995, max=2001, 
                        continuous_update=False,
                        layout=Layout(width='100%'))

def update_income(*_):
    year = year_slider.value
    id_to_income = get_id_to_income(year)
    nl_map_color_scale = nl_map.scales['color']
    nl_map_color_scale.min = min(id_to_income.values())
    nl_map_color_scale.max = max(id_to_income.values())
    nl_map.color = id_to_income
    
    
year_slider.observe(update_income, 'value')
play_button = Play(min=1995, max=2001, interval=1000, layout=Layout(width='100%'))
jslink((play_button, 'value'), (year_slider, 'value'))

VBox([
    HBox([play_button, year_slider]), 
    nl_fig,
    province_yearly_income_fig
]) 
